ITMO Computer Vision Course 2022

---

# Лабораторная работа №3

**Простые системы классификации изображений на основе сверточных нейронных сетей**



---

Перед запуском:


*   Нужно подгрузить лейблы классов - см. закомментированные строки в коде
*   Датасет для определения нужно положить в папку /samples/, которую нужно создать - тоже смотри код

Вариант 1 - AlexNet:

In [1]:
#Подгружаем данные
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

#Загружаем 
!wget https://db.rzhevskyrobotics.com/s/aszMPeLN2Y4CcQS/download/samples.zip
!unzip samples.zip

#Система автоматом создает эту ерунду, а библа пытается там прочитать фотки
!rm -R ./samples/.ipynb_checkpoints

--2023-03-13 16:09:43--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2023-03-13 16:09:43 (105 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]

--2023-03-13 16:09:43--  https://db.rzhevskyrobotics.com/s/aszMPeLN2Y4CcQS/download/samples.zip
Resolving db.rzhevskyrobotics.com (db.rzhevskyrobotics.com)... 91.227.47.147
Connecting to db.rzhevskyrobotics.com (db.rzhevskyrobotics.com)|91.227.47.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25304867 (24M) [application/zip]
Saving to: ‘samples.zip’

samples.zip        

In [3]:
#
#Основной скрипт
#
from torchvision import transforms
from torchvision import models
import torch
from PIL import Image

from torch.profiler import profile, record_function, ProfilerActivity

import os
import psutil

import time
from os import walk

#
# Функции для анализа потребляемой памяти - без библиотек(сейчас не используем)
#

# # inner psutil function
# def process_memory():
#     process = psutil.Process(os.getpid())
#     mem_info = process.memory_info()
#     return mem_info.rss
 
# # decorator function
# def profile(func):
#     def wrapper(*args, **kwargs):
 
#         mem_before = process_memory()
#         result = func(*args, **kwargs)
#         mem_after = process_memory()
#         print("{}:consumed memory: {:,}".format(
#             func.__name__,
#             mem_before, mem_after, mem_after - mem_before))
 
#         return result
#     return wrapper


#
# Функция для получения самых повторяемых меток
#
def sortList(labels_list):
  li = sorted(set(labels_list), key = lambda ele: labels_list.count(ele))
  return list(reversed(li))[:5]


#
# Основной процесс
#

### @profile
def main_process(input_batch):

  
  alexnet = models.alexnet(pretrained=True)

  alexnet.eval()

  # Читаем файл с категориями. По идее, 1000 классов
  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  #Время исполнения начало
  start = time.time()

  with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:

    #Считываем изображения
    current_item  = 1
    labels_list = []
    for current_data in input_batch:

      #print(">>>>> Item {} :".format(current_item))

      # Если можем, то заливаем на GPU
      if torch.cuda.is_available():
          current_data = current_data.to('cuda')
          alexnet.to('cuda')

      with torch.no_grad():
          output = alexnet(current_data)

      # Нормализуем скоры через софтмакс для получения вероятностей. 
      probabilities = torch.nn.functional.softmax(output[0], dim=0)
      #print(probabilities)


      # Для каждого изображения показываем топ-5
      top5_prob, top5_catid = torch.topk(probabilities, 5)
      for i in range(top5_prob.size(0)):
          labels_list.append(categories[top5_catid[i]])
          #print(categories[top5_catid[i]], top5_prob[i].item())

      current_item = current_item +1


  #Время исполнения конец
  end = time.time()
  #Печатаем время исполнения
  print("Time consuption: {} s".format((end-start)))
  print(sortList(labels_list))
  print("-"*10)
  print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


#
# НАЧАЛО СКРИПТА
#

input_batch = []

#Пока без проверки :)
filenames = next(walk("./samples"), (None, None, []))[2]

#Загружаем изображения, нормируем их, загружаем в микробатчик
for image_name in filenames:
  input_image = Image.open("./samples/"+image_name)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)  
  current_batch = input_tensor.unsqueeze(0)
  input_batch.append(current_batch)

main_process(input_batch)


Time consuption: 3.368673086166382 s
['red fox', 'kit fox', 'grey fox', 'dhole', 'red wolf']
----------
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.16%       3.979ms         0.16%       3.979ms       7.235us      92.50 Mb      92.50 Mb           550  
    aten::max_pool2d_with_indices         5.70%     137.599ms         5.70%     137.599ms     917.327us      50.40 Mb      50.40 Mb           150  
                      aten::addmm        41.44%        1.000s        41.54%        1.003s       6.685ms       1.75 Mb       1.75 Mb           150  
        

Теперь ResNet101, тоже тренировали на ImageNet
https://pytorch.org/hub/pytorch_vision_resnet/ 

In [5]:
#
#Основной скрипт
#
from torchvision import transforms
from torchvision import models
import torch
from PIL import Image

from torch.profiler import profile, record_function, ProfilerActivity

import os
import psutil

import time
from os import walk

#
# Функции для анализа потребляемой памяти - без библиотек(сейчас не используем)
#

# # inner psutil function
# def process_memory():
#     process = psutil.Process(os.getpid())
#     mem_info = process.memory_info()
#     return mem_info.rss
 
# # decorator function
# def profile(func):
#     def wrapper(*args, **kwargs):
 
#         mem_before = process_memory()
#         result = func(*args, **kwargs)
#         mem_after = process_memory()
#         print("{}:consumed memory: {:,}".format(
#             func.__name__,
#             mem_before, mem_after, mem_after - mem_before))
 
#         return result
#     return wrapper


#
# Функция для получения самых повторяемых меток
#
def sortList(labels_list):
  li = sorted(set(labels_list), key = lambda ele: labels_list.count(ele))
  return list(reversed(li))[:5]


#
# Основной процесс
#

### @profile
def main_process(input_batch):

  
  resnet = models.resnet101(pretrained=True)

  resnet.eval()

  # Читаем файл с категориями. По идее, 1000 классов
  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  #Время исполнения начало
  start = time.time()

  with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:

    #Считываем изображения
    current_item  = 1
    labels_list = []
    for current_data in input_batch:

      #print(">>>>> Item {} :".format(current_item))

      # Если можем, то заливаем на GPU
      if torch.cuda.is_available():
          current_data = current_data.to('cuda')
          resnet.to('cuda')

      with torch.no_grad():
          output = resnet(current_data)

      # Нормализуем скоры через софтмакс для получения вероятностей. 
      probabilities = torch.nn.functional.softmax(output[0], dim=0)
      #print(probabilities)


      # Для каждого изображения показываем топ-5
      top5_prob, top5_catid = torch.topk(probabilities, 5)
      for i in range(top5_prob.size(0)):
          labels_list.append(categories[top5_catid[i]])
          #print(categories[top5_catid[i]], top5_prob[i].item())

      current_item = current_item +1


  #Время исполнения конец
  end = time.time()
  #Печатаем время исполнения
  print("Time consuption: {} s".format((end-start)))
  print(sortList(labels_list))
  print("-"*10)
  print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


#
# НАЧАЛО СКРИПТА
#

input_batch = []

#Пока без проверки :)
filenames = next(walk("./samples"), (None, None, []))[2]

#Загружаем изображения, нормируем их, загружаем в микробатчик
for image_name in filenames:
  input_image = Image.open("./samples/"+image_name)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)  
  current_batch = input_tensor.unsqueeze(0)
  input_batch.append(current_batch)

main_process(input_batch)

Time consuption: 45.71519947052002 s
['grey fox', 'red fox', 'kit fox', 'coyote', 'red wolf']
----------
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.40%     122.134ms         0.40%     122.134ms       3.193us       3.72 Gb       3.72 Gb         38250  
                    aten::resize_         0.08%      25.254ms         0.08%      25.254ms       7.539us       1.72 Gb       1.72 Gb          3350  
       aten::_slow_conv2d_forward        67.47%       20.767s        67.65%       20.823s       6.216ms       2.33 Gb     620.54 Mb          3350  
    ate

Теперь пробуем VGG16:

In [7]:
#
#Основной скрипт
#
from torchvision import transforms
from torchvision import models
import torch
from PIL import Image

from torch.profiler import profile, record_function, ProfilerActivity

import os
import psutil

import time
from os import walk

#
# Функции для анализа потребляемой памяти - без библиотек(сейчас не используем)
#

# # inner psutil function
# def process_memory():
#     process = psutil.Process(os.getpid())
#     mem_info = process.memory_info()
#     return mem_info.rss
 
# # decorator function
# def profile(func):
#     def wrapper(*args, **kwargs):
 
#         mem_before = process_memory()
#         result = func(*args, **kwargs)
#         mem_after = process_memory()
#         print("{}:consumed memory: {:,}".format(
#             func.__name__,
#             mem_before, mem_after, mem_after - mem_before))
 
#         return result
#     return wrapper


#
# Функция для получения самых повторяемых меток
#
def sortList(labels_list):
  li = sorted(set(labels_list), key = lambda ele: labels_list.count(ele))
  return list(reversed(li))[:5]


#
# Основной процесс
#

### @profile
def main_process(input_batch):

  
  vgg16 = models.vgg16(pretrained=True)

  vgg16.eval()

  # Читаем файл с категориями. По идее, 1000 классов
  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  #Время исполнения начало
  start = time.time()

  with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:

    #Считываем изображения
    current_item  = 1
    labels_list = []
    for current_data in input_batch:

      #print(">>>>> Item {} :".format(current_item))

      # Если можем, то заливаем на GPU
      if torch.cuda.is_available():
          current_data = current_data.to('cuda')
          vgg16.to('cuda')

      with torch.no_grad():
          output = vgg16(current_data)

      # Нормализуем скоры через софтмакс для получения вероятностей. 
      probabilities = torch.nn.functional.softmax(output[0], dim=0)
      #print(probabilities)


      # Для каждого изображения показываем топ-5
      top5_prob, top5_catid = torch.topk(probabilities, 5)
      for i in range(top5_prob.size(0)):
          labels_list.append(categories[top5_catid[i]])
          #print(categories[top5_catid[i]], top5_prob[i].item())

      current_item = current_item +1


  #Время исполнения конец
  end = time.time()
  #Печатаем время исполнения
  print("Time consuption: {} s".format((end-start)))
  print(sortList(labels_list))
  print("-"*10)
  print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


#
# НАЧАЛО СКРИПТА
#

input_batch = []

#Пока без проверки :)
filenames = next(walk("./samples"), (None, None, []))[2]

#Загружаем изображения, нормируем их, загружаем в микробатчик
for image_name in filenames:
  input_image = Image.open("./samples/"+image_name)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)  
  current_batch = input_tensor.unsqueeze(0)
  input_batch.append(current_batch)

main_process(input_batch)

Time consuption: 27.870499849319458 s
['red fox', 'kit fox', 'grey fox', 'dhole', 'coyote']
----------
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.13%      32.040ms         0.13%      32.040ms      23.733us       2.52 Gb       2.52 Gb          1350  
    aten::max_pool2d_with_indices         5.81%        1.427s         5.81%        1.427s       5.709ms     875.49 Mb     875.49 Mb           250  
                    aten::resize_         0.00%     598.000us         0.00%     598.000us      11.960us       3.83 Mb       3.83 Mb            50  
         